In [9]:
def vectorfield(w, t, p, store):
    """
    Defines the differential equations for the coupled spring-mass system.

    Arguments:
        w :  vector of the state variables:
                  w = [x1,y1,x2,y2]
        t :  time
        p :  vector of the parameters:
                  p = [m1,m2,k1,k2,L1,L2,b1,b2]
    """
    x1, y1, x2, y2 = w
    m1, m2, k1, k2, L1, L2, = p
    
    # Friction coefficients
    b1 = 0.8 * x1
    
    if (x2 >= 1.0):
        b2 = 20*x2
    else:
        b2 = 1.0
    
    store.append({'time': t, 'coefs': [b1, b2]})
    
    # Create f = (x1',y1',x2',y2'):
    f = [y1,
         (-b1 * y1 - k1 * (x1 - L1) + k2 * (x2 - x1 - L2)) / m1,
         y2,
         (-b2 * y2 - k2 * (x2 - x1 - L2)) / m2]
    
    #print('b1', b1)
    #print('b2', b2)
    
    return f



In [2]:
# Use ODEINT to solve the differential equations defined by the vector field
from scipy.integrate import odeint

# Parameter values
# Masses:
m1 = 1.0
m2 = 1.5
# Spring constants
k1 = 8.0
k2 = 40.0
# Natural lengths
L1 = 0.5
L2 = 1.0

# Initial conditions
# x1 and x2 are the initial displacements; y1 and y2 are the initial velocities
x1 = 0.5
y1 = 0.0
x2 = 2.25
y2 = 0.0

# ODE solver parameters
abserr = 1.0e-8
relerr = 1.0e-6
stoptime = 10.0
numpoints = 1001

# Create the time samples for the output of the ODE solver.
t = [stoptime * float(i) / (numpoints - 1) for i in range(numpoints)]

# Pack up the parameters and initial conditions:
p = [m1, m2, k1, k2, L1, L2]
w0 = [x1, y1, x2, y2]


In [3]:
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt

In [4]:
numpoints = 1001
t0 = np.linspace(0, stoptime, numpoints)

In [6]:
#sol1 = odeint(vectorfield, w0, t0, args=(p,), atol=abserr, rtol=relerr)

In [10]:
c = []
sol1 = odeint(vectorfield, w0, t0, args=(p, c), atol=abserr, rtol=relerr)

In [11]:
c

[{'time': 0.0, 'coefs': [0.4, 45.0]},
 {'time': 3.331483023263848e-07, 'coefs': [0.4, 45.0]},
 {'time': 3.331483023263848e-07,
  'coefs': [0.4000000000026638, 44.999999999955605]},
 {'time': 6.662966046527696e-07,
  'coefs': [0.4000000000053275, 44.99999999991122]},
 {'time': 6.662966046527696e-07,
  'coefs': [0.40000000000799124, 44.99999999986683]},
 {'time': 0.0001385450759485236,
  'coefs': [0.4000002303394594, 44.99999616108455]},
 {'time': 0.0001385450759485236,
  'coefs': [0.40000023033316784, 44.99999616894787]},
 {'time': 0.00027642385529239445,
  'coefs': [0.40000091689870376, 44.999984749624176]},
 {'time': 0.00027642385529239445,
  'coefs': [0.40000091688595935, 44.999984765320804]},
 {'time': 0.00041430263463626527,
  'coefs': [0.40000205965321767, 44.9999658046072]},
 {'time': 0.00041430263463626527,
  'coefs': [0.40000205964015073, 44.99996582024386]},
 {'time': 0.0010899089129548108,
  'coefs': [0.40001425282474457, 44.9997648154195]},
 {'time': 0.0010899089129548108,
 

In [ ]:
def func(t, x0, beta):
    return vectorfield(x0, t, beta)

In [ ]:
sol2 = solve_ivp(func, [t0[0], t0[-1]], w0, args=(p,), t_eval=t0, method='LSODA')

In [ ]:
def coefs(x):
    # Ensure signature:
    if isinstance(x, (list, tuple)):
        x = np.array(x)
    if len(x.shape) == 1:
        x = x.reshape(1, -1)
    # Compute coefficients
    b1 = 0.8 * x[:,0]
    b2 = 20. * x[:,2]
    q2 = (x[:,2] < 1.0)
    b2[q2] = 1.0
    return np.stack([b1, b2]).T

In [ ]:
def system(t, w, p):
    x1, y1, x2, y2 = w
    m1, m2, k1, k2, L1, L2, = p
    b = coefs(w)
    return [
        y1, (-b[:,0] * y1 - k1 * (x1 - L1) + k2 * (x2 - x1 - L2)) / m1,
        y2, (-b[:,1] * y2 - k2 * (x2 - x1 - L2)) / m2,
    ]

In [ ]:
sol3 = solve_ivp(system, [t0[0], t0[-1]], w0, args=(p,), t_eval=t0, method='LSODA')

In [ ]:
fig, axe = plt.subplots()
axe.plot(t0, sol3.y.T)
axe.set_title("Dynamic System: Coupled Spring Mass")
axe.set_xlabel("Time, $t$")
axe.set_ylabel("System Coordinates, $x_i(t)$")
axe.legend([r'$x_%d$' % i for i in range(sol3.y.T.shape[1])])
axe.grid()

In [ ]:
fig, axes = plt.subplots(2,1, sharex=True, sharey=False)
axes[0].plot(t0, coefs(sol3.y.T)[:,0])
axes[1].plot(t0, coefs(sol3.y.T)[:,1])
axes[0].set_title("Dynamic System: Coupled Spring Mass")
axes[1].set_xlabel("Time, $t$")
axes[0].set_ylabel("$b_0(t)$")
axes[1].set_ylabel("$b_1(t)$")
for i in range(2):
    axes[i].grid()

In [ ]:
sol1 - sol2.y.T

In [ ]:
sol1 - sol3.y.T

In [ ]:
sol2.y.T - sol3.y.T

In [ ]:
sol3